# A Few Shot Sample Examples

In [1]:
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
deployment_name = os.getenv('DEPLOYMENT_NAME')

# A Few Shot Learning

In [2]:
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

🎉,👏,😍,✨,👌


# Chain of Thought

Experiment results demonstrate Zero-shot-CoT using a single prompt template significantly outperforms zero-shot LLM performance on diverse benchmark reasoning tasks. Without any hand-crafted few-shot examples, e.g., increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with the large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

In [3]:
def call_openai(prompt, temperature=1, max_token=60):
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
            max_tokens=max_token,
            temperature = temperature
    )
    return response.choices[0].message.content

In [4]:
# This prompt gets the wrong answer - ZERO_SHOT Prompt

system_prompt = """Answer the following questions:"""
quiz = """The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning.
They sold 93 loaves in the morning and 39 loaves in the afternoon.
A grocery store returned 6 unsold loaves.
How many loaves of bread did they have left?"""

prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

To determine how many loaves of bread the bakers had left, let's break it down step by step:

1. **Total loaves baked**:  
   The bakers started with **200 loaves**.

2. **Loaves sold in the morning**:  
   They sold **93 loaves** in the morning.

3. **Loaves sold in the afternoon**:  
   They sold **39 loaves** in the afternoon.

   Total loaves sold = \( 93 + 39 = 132 \).

4. **Loaves returned by the grocery store**:  
   The grocery store returned **6 loaves**.

5. **Remaining loaves**:  
   To find the remaining loaves, subtract the total loaves sold from the total baked, then add the returned loaves:  
   \[
   200 - 132 + 6 = 74
   \]

So, the bakers had **74 loaves of bread


In [5]:
# Still wrong answer with few-shot learning

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

To calculate how many loaves of bread the bakers had left, let's break it down step by step:

1. **Total loaves baked**: 200 loaves.
2. **Loaves sold in the morning**: 93 loaves.
3. **Loaves sold in the afternoon**: 39 loaves.
   - Total loaves sold = \( 93 + 39 = 132 \) loaves.
4. **Loaves returned by the grocery store**: 6 loaves.
   - Remaining loaves = \( 200 - 132 + 6 = 74 \) loaves.

**Final Answer**: The bakers had **74 loaves of bread** left.


In [6]:
# With CoT, the answer is correct

cot_message = " Understand the sentences well and let’s think step by step."

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz + cot_message}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

Let's break this problem into smaller steps and solve it step by step:

### Step 1: Total loaves baked
The bakers started with **200 loaves of bread**.

### Step 2: Loaves sold in the morning
They sold **93 loaves** in the morning. Subtract this from the total:
\[
200 - 93 = 107
\]
After the morning sales, they had **107 loaves** left.

### Step 3: Loaves sold in the afternoon
They sold **39 loaves** in the afternoon. Subtract this from the remaining loaves:
\[
107 - 39 = 68
\]
After the afternoon sales, they had **68 loaves** left.

### Step 4: Grocery store returned loaves
A grocery store returned **6 unsold loaves**. Add these back to the remaining loaves:
\[
68 + 6 = 74
\]
After the return, they


In [7]:
# PROMPT_FEW_SHOT_CoT

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."""},
    {"role": "user", "content": quiz + cot_message}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

Let's break this problem down step by step:

1. **Start with the total number of loaves baked**:  
   The bakers baked **200 loaves** on Monday morning.

2. **Subtract the loaves sold in the morning**:  
   They sold **93 loaves** in the morning.  
   \( 200 - 93 = 107 \) loaves remain.

3. **Subtract the loaves sold in the afternoon**:  
   They sold **39 loaves** in the afternoon.  
   \( 107 - 39 = 68 \) loaves remain.

4. **Add the loaves returned by the grocery store**:  
   A grocery store returned **6 unsold loaves**.  
   \( 68 + 6 = 74 \).

### Final Answer:  
The bakers had **74 loaves of bread** left.


# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)

Provide knowledge, turn knowledge questions into reasoning. In general, more knowledge results in better outcomes.

3 Contributing factors:

(i) The quality of knowledge, 

(ii) The quantity of knowledge, where performance improves with more knowledge statements, and 

(iii) The strategy for integrating knowledge during inference

In [8]:
common_question = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""

prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

**False.**  

In golf, the objective is to achieve the **lowest score** possible, not the highest. Players aim to complete the course in the fewest number of strokes, and the player with the lowest total score at the end of the game wins.


In [9]:
common_question = """A tripod is a kind of easel
How many legs does an easel have?
"""
prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

An easel typically has **three legs**, similar to a tripod. The three legs provide stability and support for holding items like canvases or boards. However, some easels, particularly studio easels, may have four legs or a different design for added stability.


## Check out the following 2 examples

In [10]:
# High confidence answer
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.
Explain and Answer: 
"""
prompt=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

No, the objective of golf is not to get a higher point total than others. In fact, it is the opposite. The goal in golf is to complete a round (typically 18 holes) using the fewest number of strokes possible. Each stroke counts as one point, and the player with the lowest total score at the end of the round is the winner. Golf is a game where minimizing your score is the key to success, unlike many other sports where a higher score is desirable.


In [11]:
# Low confidence answer.
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: Golf is a precision club-and-ball sport in which competing players (or golfers) use several types of clubs to hit a ball into a series of holes on a course using the fewest number of strokes. A goal is highest rated with the lowest score calculated by adding up the total number of strokes taken on each hole. The player with the lowest score wins the game.
Explain and Answer: 
"""
prompt=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

No, the statement is incorrect. In golf, the objective is not to achieve a higher point total than others but rather to achieve the **lowest score**. Golf is a sport where players aim to complete the course using the fewest number of strokes possible. Each stroke is counted as a point, and the total score is calculated by adding up the strokes taken on each hole. The player with the **lowest total score** at the end of the game is declared the winner. 

So, the
